# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [1]:
# ----------- ШАГ 1.1: Скачивание датасета -----------
# 📦 Цель: загрузить данные из Kaggle

# Для локального запуска загрузите kaggle.json в ~/.kaggle/kaggle.json и дайте права:
# !chmod 600 ~/.kaggle/kaggle.json

# Для запуска в colab

# from google.colab import files
# uploaded = files.upload('/root/.kaggle')
# !chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -p ../data -c m5-forecasting-accuracy --force

  0%|                                               | 0.00/45.8M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:00<00:00, 2.97GB/s]


In [2]:
# ----------- ШАГ 1.2: Распаковка архива -----------
# 📁 Цель: разархивировать m5-forecasting-accuracy.zip

import zipfile, os

zip_path = "../data/m5-forecasting-accuracy.zip"
extract_dir = "../data"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Архив распакован в:", extract_dir)

✅ Архив распакован в: ../data


In [3]:
# ----------- Шаг 2.1: Загрузка данных ----------- #
# 📥 Цель: считать три ключевых файла и сразу снизить объём памяти

import pandas as pd
import numpy as np
import os

# Типы для категориальных колонок в sales
dtype_sales = {col: 'category' for col in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']}

# Загружаем данные
print("📥 Загружаем данные...")
sales = pd.read_csv(f"{extract_dir}/sales_train_validation.csv", dtype=dtype_sales)
calendar = pd.read_csv(f"{extract_dir}/calendar.csv", parse_dates=['date'])
prices = pd.read_csv(f"{extract_dir}/sell_prices.csv")

print(f"✅ Форматы считаны. Размеры:")
print(f"sales    : {sales.shape}")
print(f"calendar : {calendar.shape}")
print(f"prices   : {prices.shape}")

📥 Загружаем данные...
✅ Форматы считаны. Размеры:
sales    : (30490, 1919)
calendar : (1969, 14)
prices   : (6841121, 4)


In [4]:
# ----------- Шаг 2.2: Оптимизация числовых колонок ----------- #
# 📦 Цель: привести числовые столбцы к оптимальным типам и посчитать экономию памяти

def mem_usage(df):
    return round(df.memory_usage(deep=True).sum() / 1024**2, 1)

mem_before = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

# sales: целые числа
sales_int = sales.select_dtypes(include='int').columns
sales[sales_int] = sales[sales_int].astype(np.uint16)

# calendar: float32 и int16
calendar_float = calendar.select_dtypes(include='float').columns
calendar[calendar_float] = calendar[calendar_float].astype(np.float32)

calendar_int = calendar.select_dtypes(include='int').columns
calendar[calendar_int] = calendar[calendar_int].astype(np.int16)

# prices: int и float
prices['wm_yr_wk'] = prices['wm_yr_wk'].astype(np.int16)
prices['sell_price'] = prices['sell_price'].astype(np.float32)

mem_after = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

print("📊 Оптимизация памяти:")
for name in mem_before:
    before = mem_before[name]
    after = mem_after[name]
    delta = before - after
    perc = delta / before * 100 if before > 0 else 0
    print(f"▪ {name:8}: {before:>6} ➜ {after:>6} MiB ({perc:+.1f}% экономия)")

📊 Оптимизация памяти:
▪ sales   :  448.7 ➜  115.0 MiB (+74.4% экономия)
▪ calendar:    0.6 ➜    0.5 MiB (+16.7% экономия)
▪ prices  :  853.1 ➜  787.9 MiB (+7.6% экономия)


In [5]:
# ----------- Шаг 2.3: Оптимизация категориальных колонок ----------- #
# 🏷 Цель: перевести строковые признаки в category и сократить память

cat_cols_sales = ['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id']
cat_cols_calendar = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
cat_cols_prices = ['item_id', 'store_id']

mem_before = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

for col in cat_cols_sales:
    sales[col] = sales[col].astype('category')

for col in cat_cols_calendar:
    if col in calendar.columns:
        calendar[col] = calendar[col].astype('category')

for col in cat_cols_prices:
    prices[col] = prices[col].astype('category')

mem_after = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

print("🏷 Оптимизация категориальных признаков:")
for name in mem_before:
    before = mem_before[name]
    after = mem_after[name]
    delta = before - after
    perc = delta / before * 100 if before > 0 else 0
    print(f"▪ {name:8}: {before:>6} ➜ {after:>6} MiB ({perc:+.1f}% экономия)")


🏷 Оптимизация категориальных признаков:
▪ sales   :  115.0 ➜  115.0 MiB (+0.0% экономия)
▪ calendar:    0.5 ➜    0.2 MiB (+60.0% экономия)
▪ prices  :  787.9 ➜   59.0 MiB (+92.5% экономия)


In [6]:
# ---------- Шаг 2.4: Проверка целостности данных ---------- #
# 🧪 Цель: убедиться, что данные остались корректными после оптимизации

print("🔍 Проверка целостности:")

# Проверяем размеры
print(f"- Размеры sales:     {sales.shape}")
print(f"- Размеры calendar:  {calendar.shape}")
print(f"- Размеры prices:    {prices.shape}")

# Проверяем базовую статистику
print("\n🧪 Примеры статистики:")
print(f"  Сумма продаж       : {sales.iloc[:, 6:].sum().sum():,.0f}")
print(f"  Уникальных дат     : {calendar['date'].nunique()}")
print(f"  Цены: min={prices['sell_price'].min()}, max={prices['sell_price'].max()}")

# Проверка типов
print("\n📋 Типы данных:")
print("sales:")
print(sales.dtypes.value_counts())

print("\ncalendar:")
print(calendar.dtypes.value_counts())

print("\nprices:")
print(prices.dtypes.value_counts())


🔍 Проверка целостности:
- Размеры sales:     (30490, 1919)
- Размеры calendar:  (1969, 14)
- Размеры prices:    (6841121, 4)

🧪 Примеры статистики:
  Сумма продаж       : 65,695,409
  Уникальных дат     : 1969
  Цены: min=0.009999999776482582, max=107.31999969482422

📋 Типы данных:
sales:
uint16      1913
category       1
category       1
category       1
category       1
category       1
category       1
Name: count, dtype: int64

calendar:
int16             7
datetime64[ns]    1
category          1
object            1
category          1
category          1
category          1
category          1
Name: count, dtype: int64

prices:
category    1
category    1
int16       1
float32     1
Name: count, dtype: int64


In [7]:
# ----------- Шаг 3.1: Приводим продажи к long-формату ----------- #
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product_info = sales[id_cols].copy()

# Удаляем метаинфо, оставляем только продажи
sales_long = sales.drop(columns=id_cols)

# Преобразуем в long-формат, id переименуем в row_id
sales_long = sales_long.reset_index().melt(
    id_vars='index',
    var_name='d',
    value_name='sales'
).rename(columns={'index': 'row_id'})

print("✅ Long-формат готов:", sales_long.shape)
display(sales_long.sample(5))


✅ Long-формат готов: (58327370, 3)


,row_id,d,sales
38358353,1933,d_1259,3
21241673,20633,d_697,0
25259733,14013,d_829,0
14609530,4820,d_480,0
11421826,18566,d_375,0


In [8]:
# ----------- Шаг 3.2: Объединяем календарь, товары, цены ----------- #

# Добавим календарные признаки по "d"
sales_long = sales_long.merge(calendar, on='d', how='left')

# Добавим товарные признаки по row_id → id
sales_long = sales_long.merge(product_info, left_on='row_id', right_index=True, how='left')

# Добавим цену по ключам
sales_long = sales_long.merge(
    prices[['store_id', 'item_id', 'wm_yr_wk', 'sell_price']],
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# Проверка
print("✅ Данные объединены. Размер:", sales_long.shape)
display(sales_long.sample(5))

# Проверим пропуски
missing_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'sell_price']
print("\n🔍 Доля пропусков:")
print(sales_long[missing_cols].isna().mean().sort_values(ascending=False))


✅ Данные объединены. Размер: (58327370, 23)


,row_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,...,snap_CA,snap_TX,snap_WI,id,item_id,dept_id,cat_id,store_id,state_id,sell_price
27143132,7032,d_891,3,2013-07-07,11324,Sunday,2,7,2013,NaN,...,1,1,0,HOUSEHOLD_1_377_CA_3_validation,HOUSEHOLD_1_377,HOUSEHOLD_1,HOUSEHOLD,CA_3,CA,8.96
26161451,1031,d_859,0,2013-06-05,11319,Wednesday,5,6,2013,NaN,...,1,1,1,HOUSEHOLD_1_476_CA_1_validation,HOUSEHOLD_1_476,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,NaN
50047688,13598,d_1642,0,2015-07-28,11526,Tuesday,4,7,2015,NaN,...,0,0,0,HOUSEHOLD_2_307_TX_1_validation,HOUSEHOLD_2_307,HOUSEHOLD_2,HOUSEHOLD,TX_1,TX,5.98
12648441,25581,d_415,2,2012-03-18,11208,Sunday,2,3,2012,NaN,...,0,0,0,HOUSEHOLD_2_093_WI_2_validation,HOUSEHOLD_2_093,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,3.27
46165446,3586,d_1515,0,2015-03-23,11508,Monday,3,3,2015,NaN,...,0,0,0,HOBBIES_2_122_CA_2_validation,HOBBIES_2_122,HOBBIES_2,HOBBIES,CA_2,CA,0.88



🔍 Доля пропусков:
sell_price    0.210869
item_id       0.000000
dept_id       0.000000
cat_id        0.000000
store_id      0.000000
dtype: float64


In [9]:
# ----------- Шаг 3.3: Обработка пропусков в ценах ----------- #
sales_long['has_price'] = sales_long['sell_price'].notna().astype('int')

print("✅ Добавлен флаг наличия цены. Пример:")
display(sales_long[['sell_price', 'has_price']].sample(5))


✅ Добавлен флаг наличия цены. Пример:


,sell_price,has_price
55485241,2.78,1
35175434,2.68,1
6822333,NaN,0
15150278,2.58,1
33488382,5.84,1


In [10]:
# ----------- Шаг 4.1: Временные признаки ----------- #
# 🧠 Цель: извлечь day, weekday, month, quarter и другие признаки из даты

# Убедимся, что date — это datetime
sales_long['date'] = pd.to_datetime(sales_long['date'])

# Временные признаки
sales_long['day']        = sales_long['date'].dt.day
sales_long['weekday']    = sales_long['date'].dt.day_name()
sales_long['weekday_num']= sales_long['date'].dt.weekday  # Monday = 0
sales_long['is_weekend'] = sales_long['weekday_num'].isin([5, 6]).astype(int)
sales_long['month']      = sales_long['date'].dt.month
sales_long['quarter']    = sales_long['date'].dt.quarter
sales_long['year']       = sales_long['date'].dt.year
sales_long['week']       = sales_long['date'].dt.isocalendar().week.astype('int16')

print("✅ Временные признаки добавлены. Пример:")
display(sales_long[['date', 'weekday', 'is_weekend', 'month', 'quarter', 'week']].sample(5))


✅ Временные признаки добавлены. Пример:


,date,weekday,is_weekend,month,quarter,week
49115506,2015-06-27,Saturday,1,6,2,26
54517639,2015-12-22,Tuesday,0,12,4,52
40388769,2014-09-14,Sunday,1,9,3,37
32736323,2014-01-06,Monday,0,1,1,2
50322486,2015-08-06,Thursday,0,8,3,32


In [11]:
# ----------- Шаг 4.2: Признаки событий и логарифм цены ----------- #
# 🎯 Цель: добавить флаг события и log-преобразование цены

# Преобразуем цену (log1p — безопасно для нулей)
sales_long['log_price'] = np.log1p(sales_long['sell_price'])

# Признак наличия события
sales_long['is_event'] = (
    sales_long['event_name_1'].notna() | sales_long['event_name_2'].notna()
).astype('int')

print("✅ Добавлены признаки is_event и log_price. Пример:")
display(sales_long[['sell_price', 'log_price', 'event_name_1', 'event_name_2', 'is_event']].sample(5))


✅ Добавлены признаки is_event и log_price. Пример:


,sell_price,log_price,event_name_1,event_name_2,is_event
24350039,17.969999,2.942859,NaN,NaN,0
57901956,5.940000,1.937302,NaN,NaN,0
38365189,1.740000,1.007958,NaN,NaN,0
9440531,8.370000,2.237513,NaN,NaN,0
25129794,2.980000,1.381282,NaN,NaN,0


In [12]:
# Доля пропусков в sell_price и log_price
print("🧮 Доля пропусков:")
print(sales_long[['sell_price', 'log_price']].isna().mean())

# Распределение флага событий
print("\n🧾 Распределение is_event:")
print(sales_long['is_event'].value_counts(normalize=True))

# Примеры с is_event == 1
print("\n🔍 Примеры строк с событиями:")
display(sales_long[sales_long['is_event'] == 1][['date', 'event_name_1', 'event_name_2', 'is_event']].sample(5))


🧮 Доля пропусков:
sell_price    0.210869
log_price     0.210869
dtype: float64

🧾 Распределение is_event:
is_event
0    0.919498
1    0.080502
Name: proportion, dtype: float64

🔍 Примеры строк с событиями:


,date,event_name_1,event_name_2,is_event
22794863,2013-02-14,ValentinesDay,NaN,1
37296969,2014-06-05,NBAFinalsStart,NaN,1
1422618,2011-03-16,LentWeek2,NaN,1
3718546,2011-05-30,MemorialDay,NaN,1
510778,2011-02-14,ValentinesDay,NaN,1


In [13]:
# ----------- Шаг 5.1: Лаги и скользящие окна ----------- #
# 🎯 Цель: добавить признаки отставания и сглаживания по продажам

group_cols = ['id']

# Добавим лаги (1, 7, 14, 28 дней)
for lag in [1, 7, 14, 28]:
    sales_long[f'lag_{lag}'] = (
        sales_long.groupby(group_cols, observed=True)['sales']
        .shift(lag)
    )

# Добавим скользящие средние (7, 14, 28 дней)
for window in [7, 14, 28]:
    sales_long[f'rolling_mean_{window}'] = (
        sales_long.groupby(group_cols, observed=True)['sales']
        .transform(lambda x: x.shift(1).rolling(window).mean())
    )

# ✅ Валидация: проверим наличие признаков и отсутствие NaN в середине
print("📊 Примеры лагов и скользящих средних:")
cols_to_check = [f'lag_{l}' for l in [1, 7, 14, 28]] + [f'rolling_mean_{w}' for w in [7, 14, 28]]
display(sales_long[cols_to_check].dropna().sample(5))

# 🧪 Проверка: сколько процентов NaN — допустимы только в начале рядов
print("\n🔎 Доля пропусков:")
print(sales_long[cols_to_check].isna().mean().sort_values(ascending=False))


📊 Примеры лагов и скользящих средних:


,lag_1,lag_7,lag_14,lag_28,rolling_mean_7,rolling_mean_14,rolling_mean_28
36959030,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
39325339,1.0,4.0,2.0,1.0,1.571429,1.857143,1.892857
28720673,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
51559382,2.0,0.0,3.0,1.0,0.857143,1.142857,1.000000
21957450,0.0,1.0,0.0,3.0,0.285714,0.285714,0.357143



🔎 Доля пропусков:
lag_28             0.014637
rolling_mean_28    0.014637
lag_14             0.007318
rolling_mean_14    0.007318
lag_7              0.003659
rolling_mean_7     0.003659
lag_1              0.000523
dtype: float64


In [14]:
# ---------- Шаг 5.2: Статистики по товарам и магазинам ---------- #
# 🧮 Цель: добавить mean, median, std, min, max по товарам и магазинам

agg_funcs = ['mean', 'median', 'std', 'min', 'max']
group_targets = {
    'id': 'id',
    'store': 'store_id'
}

for group_name, group_col in group_targets.items():
    for func in agg_funcs:
        col_name = f'{group_name}_sales_{func}'
        sales_long[col_name] = (
            sales_long.groupby(group_col, observed=True)['sales']
            .transform(func)
        )

print("✅ Добавлены агрегаты по товарам и магазинам.")
cols_added = [f'{g}_sales_{f}' for g in group_targets for f in agg_funcs]
display(sales_long[cols_added].dropna().sample(5))

# Проверка доли пропусков
print("\n🧮 Пропуски в новых признаках:")
print(sales_long[cols_added].isna().mean().sort_values(ascending=False))


✅ Добавлены агрегаты по товарам и магазинам.


,id_sales_mean,id_sales_median,id_sales_std,id_sales_min,id_sales_max,store_sales_mean,store_sales_median,store_sales_std,store_sales_min,store_sales_max
50004478,0.330371,0.0,0.700868,0,6,1.319829,0.0,4.058652,0,648
11343313,1.559331,1.0,1.588649,0,12,1.319829,0.0,4.058652,0,648
31539856,1.915839,2.0,1.713143,0,12,0.959291,0.0,3.327232,0,634
54200005,1.514898,1.0,1.932413,0,28,1.043992,0.0,3.796400,0,385
27413974,1.244119,0.0,2.019906,0,20,0.974753,0.0,2.759679,0,227



🧮 Пропуски в новых признаках:
id_sales_mean         0.0
id_sales_median       0.0
id_sales_std          0.0
id_sales_min          0.0
id_sales_max          0.0
store_sales_mean      0.0
store_sales_median    0.0
store_sales_std       0.0
store_sales_min       0.0
store_sales_max       0.0
dtype: float64


In [15]:
# ---------- Шаг 5.3: EWMA-тренды по продажам ---------- #
# 🎯 Цель: добавить экспоненциально взвешенное среднее (EWMA) по продажам

ewma_spans = [7, 14, 28]
for span in ewma_spans:
    col_name = f'ewma_{span}'
    sales_long[col_name] = (
        sales_long.groupby('id', observed=True)['sales']
        .transform(lambda x: x.shift(1).ewm(span=span).mean())
    )

print("✅ EWMA-тренды добавлены:")
display(sales_long[[f'ewma_{s}' for s in ewma_spans]].dropna().sample(5))

# Проверка доли пропусков — допустимы в начале ряда
print("\n📉 Пропуски в EWMA:")
print(sales_long[[f'ewma_{s}' for s in ewma_spans]].isna().mean().sort_values(ascending=False))


✅ EWMA-тренды добавлены:


,ewma_7,ewma_14,ewma_28
21043953,4.577209e+00,3.809558,3.212071
35099653,0.000000e+00,0.000000,0.000000
29354921,0.000000e+00,0.000000,0.000000
41327834,1.249669e-11,0.000003,0.001087
32347275,1.079880e-02,0.049815,0.085396



📉 Пропуски в EWMA:
ewma_7     0.000523
ewma_14    0.000523
ewma_28    0.000523
dtype: float64


In [ ]:
# ----------- Шаг 5.4: STL-декомпозиция (батчи + ускорение) ----------- #
# 🧠 Цель: выделить тренд и сезонность с учётом памяти и скорости

from statsmodels.tsa.seasonal import STL
from tqdm.notebook import tqdm
import numpy as np
import gc, time

# Параметры
season_length = 28
batch_size = 10000  # Подбираем по ОЗУ
ids = sales_long['id'].unique()

# Контейнеры
trend_vals = []
seasonal_vals = []
index_vals = []

print("🚀 Запускаем STL-декомпозицию:")

# Обработка батчами
for i in tqdm(range(0, len(ids), batch_size), desc="STL-батчи"):
    batch_ids = ids[i:i + batch_size]
    for id_ in batch_ids:
        y = sales_long.loc[sales_long['id'] == id_, 'sales'].values
        idx = sales_long.loc[sales_long['id'] == id_].index

        if len(y) >= 2 * season_length:
            result = STL(y, period=season_length, robust=True).fit()
            trend_vals.append(pd.Series(result.trend, index=idx))
            seasonal_vals.append(pd.Series(result.seasonal, index=idx))
            index_vals.extend(idx)

    gc.collect()

# Сохраняем как один DataFrame
stl_df = pd.DataFrame({
    'index': index_vals,
    'stl_trend': pd.concat(trend_vals),
    'stl_seasonal': pd.concat(seasonal_vals)
}).set_index('index')

# Присоединяем
sales_long = sales_long.join(stl_df, how='left')

# Проверка
print("✅ STL-декомпозиция завершена.")
display(sales_long[['sales', 'stl_trend', 'stl_seasonal']].dropna().sample(5))

print("\n🧪 Пропуски после STL:")
print(sales_long[['stl_trend', 'stl_seasonal']].isna().mean())


🚀 Запускаем STL-декомпозицию:


STL-батчи:   0%|          | 0/4 [00:00<?, ?it/s]